In [71]:
import xml.etree.ElementTree as ET
from collections import defaultdict
import glob
import re
import unicodedata
import os

In [72]:
os.path.join('data/SemCor/')

'data/SemCor/'

In [79]:
for r,d,f in os.walk('data/SemCor/'):
    for file in f:
        print(os.path.splitext(f'data/SemCor/{file}'))

('data/SemCor/semcor.data', '.xml')
('data/SemCor/semcor.gold.key', '.txt')


In [2]:
semcor = ET.parse('data/SemCor/semcor.data.xml')
gold_file = 'data/SemCor/semcor.gold.key.txt'
root = semcor.getroot()

In [3]:
semcor_gold = defaultdict(str)

with open(gold_file, 'r') as f:
    for line in f:
        s = line.split()
        if len(s) > 2:
            synset = ';'.join(s[1:])
            key = s[0]
            semcor_gold[key] = synset
        else:
            key, synset = s
            semcor_gold[key] = synset

semcor_gold.default_factory = None

In [58]:
# sentences = set()
sense_annotations = []
pos = defaultdict(int)
for text in root:
    for j, sentence in enumerate(text):
        s = ' '.join([w.text for w in sentence]).replace('filde', 'filled').replace('do [ c ] ters', 'doctors').replace('``', '"').replace('&apos;&apos;', '"').replace("\''", '"').replace('offersey', 'officers').replace('takeing', 'taking')
        sentences.add(s)
        cidx = 0
        for i, word in enumerate(sentence):
            text = word.text
            # doing my best to correct some spellings that stand out..
            if text == 'filde':
                text = 'filled'
            if text == 'do [ c ] ters':
                text = 'doctors'
            if text == 'offersey':
                text = 'officers'
            if text == '``':
                text = '"'
            if text == '&apos;&apos;' or text == "\''":
                text = '"'
            text = text.replace('takeing', 'taking')
            length = len(text.split(" "))
            cidx += length
            if word.tag == 'instance':
                pos[word.attrib['pos']] += 1
                start = cidx - length
                end = cidx
                if " ".join(s.split(" ")[start:end]) != text:
                    print(i, text, " ".join(s.split(" ")[start:end]), s)
                # word, position, pos, sentence, synset
                sense_annotations.append({'word': text, 'start': start, 'end': end, 'sense': semcor_gold[word.attrib['id']], 'lemma': word.attrib['lemma'], 'pos': word.attrib['pos'], 'sentence': s})

In [70]:
for entry in sense_annotations:
    if entry['word'] != " ".join(entry['sentence'].split(" ")[entry['start']:entry['end']]):
        print(entry)

In [65]:
sense_annotations[1]

{'word': 'been',
 'start': 4,
 'end': 5,
 'sense': 'be%2:42:03::',
 'lemma': 'be',
 'pos': 'VERB',
 'sentence': 'How long has it been since you reviewed the objectives of your benefit and service program ?'}